In [1]:
import torch
from torch.autograd import Variable
import numpy
import ctypes

In [2]:
class CItem(ctypes.Structure):
    _fields_ = [
                 ('value', ctypes.c_double)
                ]

def py_list2c_array(py_list, size):
    rowType = CItem * size
    resultType = ctypes.POINTER(CItem) * size
    result = resultType()
    a = [[0] * size for i in range(size)]
    for i in range(size):
        row = rowType()
        for j in range(size):
            row[j] = CItem()
            a = ctypes.c_double(py_list[i][j])
            row[j].value = a.value
        result[i] = ctypes.cast(row, ctypes.POINTER(CItem))
    return ctypes.cast(result, ctypes.POINTER(ctypes.POINTER(CItem)))

def arr2c_arr(arr, size):
    arrType = ctypes.c_double * size
    c_arr = arrType()
    for j in range(size):
        c_arr[j] = ctypes.c_double()
        a = ctypes.c_double(arr[0][j])
        c_arr[j] = a
    return ctypes.cast(c_arr, ctypes.POINTER(ctypes.c_double))

In [3]:
cLib = ctypes.cdll.LoadLibrary('./example-app/build/libexample-app.so')


In [4]:
cLib.summs.argtypes = [ctypes.c_int]
cLib.summs.restype = ctypes.c_int

summ = cLib.summs(5, 4)
print(summ)

9


In [5]:
size = 113452
w1 = Variable(torch.ones(1, size)*2, requires_grad = True)
w2 = Variable(torch.ones(1, size)*3, requires_grad = True)
arr1 = w1.cpu().detach().numpy()
arr2 = w2.cpu().detach().numpy()

c_array1 = arr2c_arr(arr1, size)
c_array2 = arr2c_arr(arr2, size)

cLib.tensor_sum.argtypes = [ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double), ctypes.c_int]
cLib.tensor_sum.restype = ctypes.c_double


In [6]:
loss = cLib.tensor_sum(c_array1, c_array2, ctypes.c_int(size))
print(loss)

567260.0
